In [2]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device : {device}")

device : mps


In [3]:
!pip install transformers==4.4.0
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2936 sha256=1dd1e4a50cf39ab8cdbd8a4d0227040201dafa137834ff26a0e21a95aa2d35eb
  Stored in directory: /Users/kweonminseong/Library/Caches/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [4]:
!git clone https://github.com/declare-lab/MELD.git

Cloning into 'MELD'...
remote: Enumerating objects: 487, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 487 (delta 6), reused 0 (delta 0), pack-reused 475
Receiving objects: 100% (487/487), 8.12 MiB | 10.02 MiB/s, done.
Resolving deltas: 100% (254/254), done.


In [5]:
import glob
data_path = "./MELD/data/MELD/*.csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/dev_sent_emo.csv', './MELD/data/MELD/train_sent_emo.csv', './MELD/data/MELD/test_sent_emo.csv']


In [7]:
import csv


# 데이터 저장
def split(session):
    ret_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        ret_data.append(split_session[:])
    return ret_data

for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)

    # 세션 데이터 저장할 리스트
    session_dataset = []
    session = []
    speaker_set = []

    # 실제 데이터 저장 방식
    pre_session = 'start'
    for i, line in enumerate(rdr):
        if i == 0:
            # 저장할 데이터들 index 확인
            header = line
            utt_idx = header.index('Utterance')
            speaker_idx = header.index('Speaker')
            emo_idx = header.index('Emotion')
            session_idx = header.index('Dialogue_ID')
        else:
            utt = line[utt_idx]
            speaker = line[speaker_idx]

            # Unique한 스피커로 바꾸기
            if speaker in speaker_set:
                uniq_speaker = speaker_set.index(speaker)
            else:
                speaker_set.append(speaker)
                uniq_speaker = speaker_set.index(speaker)
            emotion = line[emo_idx]
            sess = line[session_idx]

            if pre_session == 'start' or sess == pre_session:
                session.append([uniq_speaker, utt, emotion])
            else:
                # 세션 데이터 저장
                # session_dataset.append(session)
                session_dataset += split(session)
                session = [[uniq_speaker, utt, emotion]]
                speaker_set = []
            pre_session = sess

    # 마지막 세션 저장
    session_dataset += split(session)
    f.close()

In [8]:
session_dataset

[[[0,
   'Why do all you’re coffee mugs have numbers on the bottom?',
   'surprise']],
 [[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise'],
  [1,
   'Oh. That’s so Monica can keep track. That way if one on them is missing, she can be like, ‘Where’s number 27?!’',
   'anger']],
 [[0, 'Why do all you’re coffee mugs have numbers on the bottom?', 'surprise'],
  [1,
   'Oh. That’s so Monica can keep track. That way if one on them is missing, she can be like, ‘Where’s number 27?!’',
   'anger'],
  [1, "Y'know what?", 'neutral']],
 [[2, 'Come on, Lydia, you can do it.', 'neutral']],
 [[2, 'Come on, Lydia, you can do it.', 'neutral'], [0, 'Push!', 'joy']],
 [[2, 'Come on, Lydia, you can do it.', 'neutral'],
  [0, 'Push!', 'joy'],
  [0, "Push 'em out, push 'em out, harder, harder.", 'joy']],
 [[2, 'Come on, Lydia, you can do it.', 'neutral'],
  [0, 'Push!', 'joy'],
  [0, "Push 'em out, push 'em out, harder, harder.", 'joy'],
  [0, "Push 'em out, push 'em out, way out!